In [17]:
import os
import openpyxl
import numpy as np
import pandas as pd
import warnings

files = [
    "GVXL_ASM_R1",
    "GVXL_ASM_R2",
    "GVXL_ASM_OUT",
    "GVXL_ASM_DC",
    "GVXL_ASM_DC_BACK",
    "GVXL_ASM_DC_RAISE_MIDDLE_GAP",
    "GVXL_ASM_DC_RAISE",
    "GVXL_4_TGAB_ROOF",
    "GVXL_3_TGAB_ROOF",
    "GVXL_2_TGAB_ROOF",
    "GVXL_BACK_ROOF",
    "GVXL_ASM_SEPARATED_INPUT",
    "GVXL_ASM_COMMON_INPUT"
]

columns = [
    "Supplier_Type",
    "Fabriquant",
    "SAT",
    "Item",
    "SE_REVISION",
    "Description",
    "Paint_Code",
    "Material",
    "Oracle_Type",
    "Qty",
    "FROM ASSEMBLY",
    "FROM VERSION",
    "Unit Cost"
]

table = [
    ["GVXL_ASM_R1", "INPUT 1", "x", "x", "x", "x", "x"],
    ["GVXL_ASM_R2", "INPUT 2", "", "x", "", "", "x"],
    ["GVXL_ASM_OUT", "OUTPUT", "x", "x", "x", "x", "x"],
    ["GVXL_ASM_DC", "DC TGAB FRONT", "", "", "x", "", ""],
    ["GVXL_ASM_DC_BACK", "DC TGAB REAR", "", "", "", "x", "x"],
    ["GVXL_ASM_DC_RAISE_MIDDLE_GAP", "VENT RAISE", "", "", "", "", "x"],
    ["GVXL_ASM_DC_RAISE", "DC RAISE", "x", "x", "", "", ""],
    ["GVXL_4_TGAB_ROOF", "4 TGAB ROOF", "", "", "", "", "x"],
    ["GVXL_3_TGAB_ROOF", "3 TGAB ROOF", "", "x", "x", "x", ""],
    ["GVXL_2_TGAB_ROOF", "2 TGAB ROOF", "x", "", "", "", ""],
    ["GVXL_BACK_ROOF", "FLAT BACK ROOF", "", "", "x", "x", ""],
    ["GVXL_ASM_SEPARATED_INPUT", "SINGLE MAIN", "", "x", "", "", "x"],
    ["GVXL_ASM_COMMON_INPUT", "DUAL MAIN", "x", "", "x", "x", ""]
]

versions = [
    "SINGLE MAIN + DC CABLE",
    "DUAL MAIN + DC CABLE",
    "SINGLE MAIN + DC FRONT",
    "SINGLE MAIN + DC REAR",
    "DUAL MAIN + DC TGAB"
]

def find_cost_of(search_string: str):
    """
    Finds the unit cost of an item in a Bill of Materials (BOM) Excel file based on a search string.

    Args:
        search_string (str): The string to search for in the 'Item' column of the BOM.

    Returns:
        float: The unit cost of the item that matches the search string. If multiple items match, returns the first match.
    """

    filePath = 'C:/Users/SESA787052/Documents/GVXL/BOM Valorisée GVXLIOCQ-QASH01250-PROTO.xlsx'
    
    with warnings.catch_warnings(action="ignore"):
        skip = 0
        BOM = pd.read_excel(filePath, skiprows=skip)
        result_row = BOM[BOM['Item'].str.contains(search_string, na=False)]
        if result_row.empty:
            return -1
        return float(result_row['Unit Cost'].values[0])

elements = []

with warnings.catch_warnings(action="ignore"):
    for index, file in enumerate(files):
        excel_file = f'C:/Users/SESA787052/Documents/GVXL/BOM_SUB_ASM/BOM_{file}.xlsx'

        df = pd.read_excel(excel_file, sheet_name='Bom', engine='openpyxl')

        df = (df.iloc[6:]).to_numpy()
        df = np.array([sublist for sublist in df if not pd.isna(sublist[0])])

        for line in df:
            for j, asm in enumerate(table[index][2:]):
                if asm == "x":
                    elements.append(np.append(line,[file, versions[j], find_cost_of(str(line[3]))]))

# print(elements[0])
# Convert elements to DataFrame using the first row as column names
elements_df = pd.DataFrame(elements[1:], columns=columns)

# Save to Excel
output_file = 'elements_output.xlsx'
elements_df.to_excel(output_file, index=False)
print(f"Elements saved to {output_file}")
os.startfile(output_file)

Elements saved to elements_output.xlsx


In [8]:
import warnings
def find_cost_of(search_string: str):
    """
    Finds the unit cost of an item in a Bill of Materials (BOM) Excel file based on a search string.

    Args:
        search_string (str): The string to search for in the 'Item' column of the BOM.

    Returns:
        pandas.Series: A series containing the unit cost(s) of the item(s) that match the search string.
    """

    filePath = 'C:/Users/SESA787052/Documents/GVXL/BOM Valorisée GVXLIOCQ-QASH01250-PROTO.xlsx'
    
    with warnings.catch_warnings(action="ignore"):
        skip = 0
        BOM = pd.read_excel(filePath, skiprows=skip)
        result_row = BOM[BOM['Item'].str.contains(search_string, na=False)]
        return result_row['Unit Cost']
    


10    67.77
Name: Unit Cost, dtype: float64


In [1]:
from compare import compare_bom
from pprint import pprint

bom1 = "C:/Users/SESA787052/Downloads/QTOX05051 BOM Design a refaire.xlsx"
bom2 = "C:/Users/SESA787052/Downloads/QTOX05051 BOM Design in work.xlsx"

compare_bom(bom1, bom2)

{'type_changes': {"root['QTOX05051-CABLE-TRAY']['content']['QSMA82234_CABLE-TRAY-LEFT']['content']['34408455_SCR']['content']['803-6061']['SupplyType']": {'new_type': <class 'str'>,
                                                                                                                                                           'new_value': 'VISSERIE',
                                                                                                                                                           'old_type': <class 'float'>,
                                                                                                                                                           'old_value': nan},
                  "root['QTOX05051-CUBICLE']['content']['GVEAC10D8-CTO-PF-A45']['content']['GVEAC10D8-SE01']['content']['803-0984']['SupplyType']": {'new_type': <class 'str'>,
                                                                                                        